# Problem (from notebooks/api_search_recs_pkl_parquet_sqlite)
- search_recs write/read with either .sqlite/.parquet is fast
- But I forgot to account for the _~20s_ it takes to convert back _to_ search_recs from the serialized df

# Solution: save/load separate files
- feats.npy for heavy np.array cols + non_feats.parquet for normal df cols
- A little more complexity to manage, but an ok tradeoff
- Conclusion: ~1s read, ~3s write

In [ ]:
from notebooks import *
sg.init(None, skip=[sg_load.load_search_recs])
get_search_recs.cache_clear()

INFO     [23:36:33.284] [57449]  52 logging_/init_logging: {logging_yaml: None}

INFO     [23:36:33.595] [57449]  32 api.server_globals/init

DEBUG    [23:36:33.602] [57449]  75 api.server_globals/load: load_search...

INFO     [23:36:33.924] [57449]  79 api.server_globals/load: load_search (took 0.316s)

DEBUG    [23:36:33.930] [57449]  75 api.server_globals/load: load_xc_meta...

...

DEBUG    [23:36:34.390] [57449]  75 api.server_globals/load: load_d_feats...

INFO     [23:36:34.396] [57449] 110 api.server_globals/load_d_feats

INFO     [23:36:34.401] [57449]  79 api.server_globals/load: load_d_feats (took 0.005s)

WARNING  [23:36:34.406] [57449]  73 api.server_globals/load: load_search_recs [skipped]

INFO     [23:36:34.412] [57449]  34 api.server_globals/init: done

In [ ]:
kwargss = [
    dict(cache_type='hybrid', refresh=True),  # Simulate cache miss
    dict(cache_type='hybrid'),                # Simulate cache hit
]
search_recss = []
for kwargs in kwargss:
    print()
    print(f'search_recs(**{kwargs})')
    get_search_recs.cache_clear()
    search_recs = get_search_recs(**kwargs)
    search_recss.append(search_recs)
    display(search_recs
        [['xc_id', 'feat', 'f_f', 'f_p', 'background', 'background_species']]
        [2:3].reset_index(drop=True).T
        .pipe(df_assign_first,
            size=lambda df: df.T.memory_usage(deep=True),
            type=lambda df: df[0].map(lambda x: type(x).__name__),
        )
    )

search_recs(**{'cache_type': 'hybrid', 'refresh': True})

INFO     [23:37:34.371] [57449] 295 api.recs/get_search_recs

INFO     [23:37:34.385] [57449] 555 util/df_cache_hybrid: Cache miss: payloads/search_recs-version[0],audio_s[10],countries_k[na],com_names_k[ca],num_recs[None]-b89a9fc

df_cache_hybrid:compute[search_recs]:   0%|                                                      | 0/1 [00:00<?, ?it/s]

INFO     [23:37:34.394] [57449] 348 api.recs/_compute_search_recs: {len(sg.xc_meta): 35233, countries_k: na, com_names_k: ca, num_recs: null}

...

INFO     [23:37:47.374] [57449] 626 util/df_cache_hybrid: Read[feat-f_f.npy] (211.4 MB)

INFO     [23:37:47.511] [57449] 626 util/df_cache_hybrid: Read[feat-f_p.npy] (93.3 MB)

INFO     [23:37:47.578] [57449] 630 util/df_cache_hybrid: Read: join non_feats + feats

INFO     [23:37:47.750] [57449] 644 util/df_cache_hybrid: Read done (took: 1.006s)

,size,type,0
xc_id,36,int,115925
feat,104,ndarray,[ 4.455e-02 -7.034e-02 2.956e-01 -4.216e-02 -3.264e-02 -6.938e-02 9.214e-02 3.209e-03 2.126e-01 -8.443e-02 -1.446e-01 -9.171e-02 -1.280e-01 1.579e-02 -7.380e-02 -7.01...
f_f,104,ndarray,[ 4.455e-02 -7.034e-02 2.956e-01 -4.216e-02 -3.264e-02 -6.938e-02 9.214e-02 3.209e-03 2.126e-01 -8.443e-02 -1.446e-01 -9.171e-02 -1.280e-01 1.579e-02 -7.380e-02 -7.01...
f_p,104,ndarray,[2.792e-04 2.333e-03 6.595e-06 7.704e-04 4.692e-04 5.952e-03 1.236e-02 4.282e-03 2.968e-05 2.268e-03 1.762e-04 2.358e-04 3.394e-03 2.391e-03 3.478e-03 2.690e-03 3.178e-04 ...
background,80,list,['Canada Goose (Branta canadensis)']
background_species,80,list,['CANG']


In [ ]:
# TODO Turn this into unit tests
#   - Should roundtrip, should preserve category dtypes, should fail if index present
[a, b] = search_recss
pd.testing.assert_frame_equal(a, b, check_index_type=True, check_column_type=True)